In [1]:
"""
Changes from OHE2:
-Here we do not drop features according to correlation with the target
-We also try here lambda = 1e-7
"""

'\nChanges from OHE2:\n-Here we do not drop features according to correlation with the target\n-We also try here lambda = 1e-7\n'

In [13]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from Data_cleaning import *
from helpers import *
from exploratory_data_analysis import *
from preprocessing import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data\dataset\dataset")

<>:1: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
<>:1: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
C:\Users\janfo\AppData\Local\Temp\ipykernel_1464\4219252773.py:1: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
  x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data\dataset\dataset")


### Create the preprocessing function

In [4]:
def preprocess_data(X_train, Y_train, X_valid, Y_valid):
    """
    Args:
        X_train: training data balanced only (no NaN treatment or feature dropping)
        X_valid: validation data balanced only (no NaN treatment or feature dropping)

    Returns:
        X_train_preprocessed: preprocessed training data
        X_valid_preprocessed: preprocessed validation data
    """

    # Tuning parameter section
    NaN_removal_threshold = 0.2
    outliers_threshold = 0.1
    correlation_threshold = 0.9

    # Detect categorical
    X_train, keep_mask = remove_nan_features(X_train, threshold=NaN_removal_threshold)
    X_valid = X_valid[:, keep_mask]
    cat_mask, num_mask = detect_categorical_features(X_train, max_unique=10)

    # Numerical NaN substitution (compute medians from training)
    X_train, medians = impute_numerical(X_train, num_mask)
    X_valid, _ = impute_numerical(X_train, num_mask, medians)


    # Categorical NaN substitution (compute modes from training)
    X_train, modes = impute_categorical(X_train, cat_mask)
    X_valid, _ = impute_categorical(X_valid, cat_mask, modes)

    # outliers 
    outlier_mask, outlier_counts = detect_outliers(X_train, num_mask, threshold=1.5, plot=False)
    outlier_pctg = outlier_counts / X_train.shape[0]
    no_outliers_num_mask = outlier_pctg < outliers_threshold

    keep_mask = np.ones(X_train.shape[1], dtype=bool)
    num_indices = np.where(num_mask)[0]
    keep_mask[num_indices] = no_outliers_num_mask

    X_train = X_train[:, keep_mask]
    X_valid = X_valid[:, keep_mask]
    num_mask = num_mask[keep_mask]
    cat_mask = cat_mask[keep_mask]

    # Fit one-hot encoder on training data
    X_train, categories = one_hot_encode(X_train, cat_mask, drop_first=True)
    X_valid, _ = one_hot_encode(X_valid, cat_mask, categories, drop_first=True)

    cat_mask_enc, num_mask_enc = detect_categorical_features(X_train, max_unique=10)

    # standarize
    X_train, means, stds = standardize_features(X_train, num_mask_enc)
    X_valid, _, _ = standardize_features(X_valid, num_mask_enc, means, stds)

    # select correlation

    corr_y = feature_target_correlation(X_train, Y_train)
    
    corr_mat = correlation_matrix(X_train)
    to_drop = set()

    for i in range(corr_mat.shape[0]):
        for j in range(i + 1, corr_mat.shape[1]):
            if abs(corr_mat[i, j]) > correlation_threshold:
                # Compare absolute correlation with y
                if abs(corr_y[i]) < abs(corr_y[j]):
                    to_drop.add(i)
                else:
                    to_drop.add(j)

    keep_mask_inter = np.ones(X_train.shape[1], dtype=bool)
    keep_mask_inter[list(to_drop)] = False
    X_train_final = X_train[:, keep_mask_inter]
    X_valid_final = X_train[:, keep_mask_inter]

    return X_train_final, Y_train, X_valid_final, Y_valid

In [11]:
print(x_train.shape, y_train.shape)

y_norm = (y_train + 1) / 2

X_train, Y_train, X_val, Y_val, X_test, Y_test= stratified_three_way_split(x_train, y_norm, val_ratio=0.15, test_ratio=0.15, seed=0)

print(X_train.shape, Y_train.shape, X_val.shape, X_val.shape, X_test.shape, Y_test.shape)

X_train, Y_train, X_val, Y_val = preprocess_data(X_train, Y_train, X_val, Y_val)

print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

(328135, 321) (328135,)
(229695, 321) (229695,) (49220, 321) (49220, 321) (49220, 321) (49220,)
(229695, 282) (229695,) (229695, 282) (49220,)


In [15]:
best_params, best_f1, results = cross_validation_grid_search(
    y_norm, x_train,
    k=5,
    lambdas=[1e-8, 1e-7],
    pos_weights=[9.0, 9.5, 10],
    thresholds=np.linspace(0.5, 0.84, 340),
    gamma=0.5,
    max_iter=5000,
    tol=1e-8,
    seed=42,
    verbose=False,
    cleaning_function=preprocess_data
)


=== Run 1/6 | Î»=1e-08 | pos_w=9.0 ===


ValueError: operands could not be broadcast together with shapes (65627,) (262508,) 